In [1]:
%load_ext autoreload
%autoreload 2

import os
os.chdir("../../")
print(os.getcwd())

C:\Users\Milosz\thesis-recsys


In [2]:
import pandas as pd
from py2neo import Graph, Node, Relationship

from scripts.build_knowledge_graph import read_items_data, create_exploded_df,ENTITY_MAP

In [3]:
dir_art = 'data/steam'

In [4]:
with open(os.path.join(dir_art, 'knowledge_graph.pkl'), "rb") as f:
    knowledge_graph = pd.read_pickle(f)

with open(os.path.join(dir_art, 'data.pkl'), "rb") as f:
    data = pd.read_pickle(f)

In [5]:
valid_set = knowledge_graph["valid_set"]
ripple_sets_valid = knowledge_graph['ripple_sets_valid']
RELATIONS_MAP = knowledge_graph["relations_map"]
ENTITY_MAP = knowledge_graph["entity_map"]

rev_rel_map = {v:k for k,v in RELATIONS_MAP.items()}
rev_ent_map = {v:k for k,v in ENTITY_MAP.items()}

In [6]:
df = read_items_data(os.path.join("./data", 'items.csv'), os.path.join("./data", 'steam.csv'), data)

app_names_map = pd.Series(df['title'])
app_names_map.index = df['app_id'] + 1
app_names_map = app_names_map.to_dict()

In [7]:
u_id = 36

In [8]:
print(list(zip(ripple_sets_valid[0].loc[u_id]['head'].values, ripple_sets_valid[0].loc[u_id]['head'].map(app_names_map).values)))

[(24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (24, 'Fallout: New Vegas'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (29, 'Fallout 3: Game of the Year Edition'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (131, 'Warframe'), (710, 'Deep Rock Galactic'), (710, 'Deep Rock Galactic'), (710, 'Deep Rock Gala

In [9]:
print(list(zip(ripple_sets_valid[0].loc[u_id]['relation'].values, ripple_sets_valid[0].loc[u_id]['relation'].map(rev_rel_map).values)))

[(3, 'tag'), (3, 'tag'), (3, 'tag'), (0, 'developed'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (2, 'system'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (0, 'developed'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (2, 'system'), (3, 'tag'), (3, 'tag'), (3, 'tag'), (3, 'tag')]


In [10]:
print(list(zip(ripple_sets_valid[0].loc[u_id]['tail'].values, ripple_sets_valid[0].loc[u_id]['tail'].map(rev_ent_map).values)))

[(2156, 'Singleplayer'), (2181, 'Atmospheric'), (2194, 'Open World'), (1248, 'Obsidian Entertainment'), (2195, 'Sandbox'), (2292, 'Western'), (2204, 'Sci-fi'), (2183, 'RPG'), (2146, 'win'), (2172, 'Adventure'), (2164, 'Moddable'), (2187, 'Shooter'), (2216, 'Exploration'), (2181, 'Atmospheric'), (2227, 'Cult Classic'), (2195, 'Sandbox'), (2172, 'Adventure'), (1243, 'Bethesda Game Studios'), (2159, 'Replay Value'), (2183, 'RPG'), (2273, 'Free to Play'), (2170, 'Hack and Slash'), (2204, 'Sci-fi'), (2256, 'Action RPG'), (2156, 'Singleplayer'), (2426, 'Lore-Rich'), (2358, 'Parkour'), (2250, 'Massively Multiplayer'), (2175, 'Third Person'), (2187, 'Shooter'), (2181, 'Atmospheric'), (2307, 'Loot'), (2251, 'Class-Based'), (2193, 'Survival'), (2169, 'Action'), (2146, 'win'), (2383, 'Combat'), (2219, 'Space'), (2216, 'Exploration'), (2287, 'Dark')]


In [11]:
rel_names = {'developed': "developed_by", 'published': "published_by", 'system': "available_on_system",  "tag": "tagged_as"}
node_names = {'developed': "Developer", 'published': "Publisher", 'system': "System",  "tag": "Tag"}

In [12]:
all_games = {k: Node("Game", name=i) for k,i in app_names_map.items()}

In [13]:
other_nodes = {}
def add_other_nodes(x):
    other_nodes[x['relation'].iloc[0]] = x['tail'].unique()
ripple_sets_valid[0].reset_index()[['relation', 'tail']].groupby('relation').apply(add_other_nodes)

other_nodes_map = {}
for k, v in other_nodes.items():
    other_nodes_map[node_names[rev_rel_map[k]]] = [rev_ent_map[i] for i in v]
all_other_nodes = {ENTITY_MAP[vi]: Node(k, name=vi) for k,v in other_nodes_map.items() for vi in v}

In [14]:
graph = Graph("bolt://localhost:7687", user="neo4j", password="milosz1596")

In [15]:
for n in all_games.values():
    graph.create(n)
for n in all_other_nodes.values():
    graph.create(n)

In [16]:
rs0 = ripple_sets_valid[0].loc[u_id]
relationships0 = []
for _, row in rs0.iterrows():
    h = all_games[row['head']]
    r = rel_names[rev_rel_map[row['relation']]]
    t = all_other_nodes[row['tail']]
    relationships0.append(Relationship(h, r, t))

In [17]:
rs1 = ripple_sets_valid[1].loc[u_id]
relationships1 = []
for _, row in rs1.iterrows():
    h = all_other_nodes[row['head']]
    r = rel_names[rev_rel_map[row['relation']]]
    t = all_games[row['tail']]
    relationships1.append(Relationship(h, r, t))

In [18]:
# RS0 visualisations, stop and check neo4j
for r in relationships0:
    graph.create(r)

In [19]:
# RS0 and RS1 visualisations, stop and check neo4j
for r in relationships1:
    graph.create(r)

In [ ]:
# set UsetItem set as new label

# WITH ['Fallout: New Vegas', 'Fallout 3: Game of the Year Edition', 'Warframe', 'Deep Rock Galactic'] as items
# UNWIND items as itemName
# MATCH (n {name: itemName})
# SET n:UserItemSet
# RETURN n;